In [104]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# print(os.listdir("../input/test"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'train_labels.csv', 'sample_submission.csv']


In [105]:
TRAIN_DIR = "../input/train"
TEST_DIR = "../input/test"
IMG_SIZE = 32
training_data = None
testing_data = None

In [106]:
df = pd.read_csv("../input/train_labels.csv")
print(df.head())
print(df.tail())

   name  invasive
0     1         0
1     2         0
2     3         1
3     4         0
4     5         1
      name  invasive
2290  2291         1
2291  2292         1
2292  2293         1
2293  2294         1
2294  2295         1


# Preparing the training data

In [107]:
import cv2
import os
from random import shuffle
from tqdm import tqdm_notebook

def create_train_data():
    training_data = []
    for img in tqdm_notebook(os.listdir(TRAIN_DIR)):
        label = df.invasive[int(img.split('.')[0])-1]
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        # Normalizing the data and appending to training_data list
        training_data.append([np.array(img/255),label])
    shuffle(training_data)
    return training_data

def create_test_data():
    testing_data = []
    for img in tqdm_notebook(os.listdir(TEST_DIR)):
        name = img.split('.')[0]
        path = os.path.join(TEST_DIR, img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([name, np.array(img)/255])
    return testing_data
    
if training_data is None:
    training_data = create_train_data()
    
if testing_data is None:
    testing_data = create_test_data()

A Jupyter Widget

A Jupyter Widget

# Train a CNN

## Importing required Libraries

In [125]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout

In [126]:
# Initialising the CNN
batch_size = 256
keep_prob = 0.7
classifier = Sequential()

In [127]:
# Step 1 - Convolution
classifier.add(Convolution2D(64, 3, 3, input_shape = (IMG_SIZE, IMG_SIZE , 3), activation = 'relu'))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  


In [128]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [129]:
# Adding a second convolutional layer
classifier.add(Convolution2D(64, 3, 3, activation = 'relu'))
classifier.add(Dropout(keep_prob))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(keep_prob))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  


In [130]:
# Step 3 - Flattening
classifier.add(Flatten())

In [131]:
# Step 4 - Full connection
classifier.add(Dense(activation='relu', units=128))
classifier.add(Dense(activation='sigmoid', units=1))

In [132]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Creating train and validation split

In [133]:
split_size = 1800
train = training_data[:split_size]
validation = training_data[split_size:]

In [134]:
# train_x = np.array(train).transpose()[0]
# train_y = np.array(train).transpose()[1]

train_x = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
# train_y = [i[1] for i in train]
train_y = np.array(train).transpose()[1]

validation_x = np.array([i[0] for i in validation]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
validation_y = np.array(validation).transpose()[1]

testing_x = np.array([i[1] for i in testing_data]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
testing_name = np.array([i[0] for i in testing_data])
# testing_x = np.array(testing_data)
# testing_x = np.array(i for i in testing_data).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [135]:
print(train_x.shape)
print(validation_x.shape)
print(testing_x.shape)

(1800, 32, 32, 3)
(495, 32, 32, 3)
(1531, 32, 32, 3)


In [139]:
# Train the model, iterating on the data in batches of 32 samples
classifier.fit(train_x, train_y, epochs=110, batch_size=batch_size)

Epoch 1/110
1800/1800 [==============================] - 0s 159us/step - loss: 0.2750 - acc: 0.8822
Epoch 2/110
1800/1800 [==============================] - 0s 130us/step - loss: 0.2729 - acc: 0.8889
Epoch 3/110
1800/1800 [==============================] - 0s 122us/step - loss: 0.2562 - acc: 0.8950
Epoch 4/110
1800/1800 [==============================] - 0s 117us/step - loss: 0.2800 - acc: 0.8739
Epoch 5/110
1800/1800 [==============================] - 0s 115us/step - loss: 0.2765 - acc: 0.8817
Epoch 6/110
1800/1800 [==============================] - 0s 117us/step - loss: 0.2725 - acc: 0.8817
Epoch 7/110
1800/1800 [==============================] - 0s 114us/step - loss: 0.2555 - acc: 0.8906
Epoch 8/110
1800/1800 [==============================] - 0s 114us/step - loss: 0.2661 - acc: 0.8922
Epoch 9/110
1800/1800 [==============================] - 0s 115us/step - loss: 0.2675 - acc: 0.8856
Epoch 10/110
1800/1800 [==============================] - 0s 119us/step - loss: 0.2569 - acc: 0.8956

1800/1800 [==============================] - 0s 117us/step - loss: 0.2010 - acc: 0.9122
Epoch 83/110
1800/1800 [==============================] - 0s 116us/step - loss: 0.2087 - acc: 0.9189
Epoch 84/110
1800/1800 [==============================] - 0s 118us/step - loss: 0.2732 - acc: 0.8878
Epoch 85/110
1800/1800 [==============================] - 0s 119us/step - loss: 0.2285 - acc: 0.9061
Epoch 86/110
1800/1800 [==============================] - 0s 120us/step - loss: 0.2125 - acc: 0.9150
Epoch 87/110
1800/1800 [==============================] - 0s 120us/step - loss: 0.2188 - acc: 0.9139
Epoch 88/110
1800/1800 [==============================] - 0s 120us/step - loss: 0.2256 - acc: 0.9111
Epoch 89/110
1800/1800 [==============================] - 0s 119us/step - loss: 0.2262 - acc: 0.9050
Epoch 90/110
1800/1800 [==============================] - 0s 121us/step - loss: 0.2174 - acc: 0.9133
Epoch 91/110
1800/1800 [==============================] - 0s 122us/step - loss: 0.2109 - acc: 0.9144
Epo

In [140]:
score = classifier.evaluate(validation_x, validation_y, batch_size=batch_size)

495/495 [==============================] - 0s 156us/step


In [141]:
score

[0.3309219675232666, 0.8585858530468411]

In [168]:
prediction_prob = list(classifier.predict(testing_x).transpose())[0]
prediction_classes = classifier.predict_classes(testing_x)

In [172]:
print(type(prediction_prob))
print(prediction_prob)
print(type(prediction_classes))
print(prediction_classes)

<class 'numpy.ndarray'>
[0.9966627  0.74194604 0.9985933  ... 0.93569285 0.05101522 0.02971616]
<class 'numpy.ndarray'>
[[1]
 [1]
 [1]
 ...
 [1]
 [0]
 [0]]


In [174]:
print("name,invasive")
for i,j in zip(testing_name, prediction_prob):
    print(str(i)+","+str(j))

name,invasive
513,0.9966627
39,0.74194604
1284,0.9985933
1231,0.02373377
1118,0.011558203
1061,0.014073847
747,0.022275424
1230,0.38609597
150,0.033833917
1074,0.99978346
364,0.99940395
1148,0.114470914
653,0.3310578
1109,0.98434675
199,0.9030275
1489,0.93786913
428,0.9983594
146,0.87537926
281,0.78984606
865,0.054412823
129,0.02401157
1135,0.021135464
758,0.95944643
552,0.11754381
1395,0.44369784
1159,0.044721562
1298,0.028810307
1299,0.0015966912
373,0.037539255
930,0.0072777104
901,0.1497237
1183,0.02247154
1215,0.04333833
1524,0.65301234
665,0.99056655
445,0.3154392
1125,0.20455573
892,0.9875202
1375,0.0307421
584,0.15971853
366,0.022664905
565,0.14158474
1092,0.4749559
165,0.9574455
343,0.9637906
1447,0.3375096
1217,0.01395638
221,0.9975949
1014,0.06495327
475,0.96181464
1184,0.07330426
470,0.191668
666,0.024032334
527,0.99744886
1393,0.32438624
838,0.1574513
223,0.98705715
1026,0.13501671
557,0.1530556
86,0.037600394
1036,0.99846184
673,0.008157969
1010,0.06275926
1094,0.02939101